------------------- OBJECTIVES SESSION 3 -------------------<br/>
- Find all the values for from worldbank website
- Once you have it, fill it inside the initial dataframe
- Display the dataframe that contains the entire values

------------------- FIRST PART -------------------<br/>
- Find the iso code for each country and add it in the dataframe

In [1]:
import pandas as pd
import requests as rq

In [2]:
df = pd.read_excel('Data-sheet-STUDENTS.xlsx',header=2,index_col=0)

In [3]:
#create a table that will contain all the iso_codes of each countries
iso_codes = {}    

In [4]:
 #create the loop to only have the names of the countries
for index, name in df['Country (according to CN Gov):'].items(): 
    if name in iso_codes:
        #means that if the name already exists then it'll pass the step
        continue    

    #remove the error of France (Not BRI) / Thailiand / USA...
    cleaned_name = name.split('(')[0].strip()    
    if cleaned_name == 'Thailiand':
        cleaned_name = 'Thailand'
    if cleaned_name == 'The United Arab Emirates':
        cleaned_name = 'United Arab Emirates'
    
    #don't forget to change into cleaned_name    
    url = 'https://restcountries.eu/rest/v2/name/' + cleaned_name 
    #create the fonction request to obtain the isocodes
    r = rq.get(url)  
    if r.status_code == 200:
        #check the status code if 200 it's success
        data = r.json() 
        #enter inside the table and go to the first column and then to the parameter
        country_code = data[0]['alpha3Code']   
        if country_code == 'IOT':
            country_code = 'IND'
        #put the code inside the table
        iso_codes[name] = country_code   
    else:
        print('ERROR',r.status_code,r.text)
        print(name)

In [5]:
df['isocode'] = df['Country (according to CN Gov):'].apply(lambda x: iso_codes[x])

In [6]:
df[['isocode','Country (according to CN Gov):']]

,isocode,Country (according to CN Gov):
1,AFG,Afghanistan
2,ALB,Albania
3,ARM,Armenia
4,AZE,Azerbaijan
5,BHR,Bahrain
6,BGD,Bangladesh
7,BLR,Belarus
8,BTN,Bhutan
9,BIH,Bosnia and Herzegovina
10,BRN,Brunei


------------------- THIRD PART -------------------<br/>
                                           
- Create a table that contains all the URL needed<br/>
- We select the concern row of the dataframe<br/>
- If the url of the column contains 'data.wordbank', it'll put the url inside a table<br/>
- Now we have all the last part of the URL needed and we create a loop like for the isocodes in adding the tax code<br/>
- All the results are inside a table that we need to import it inside the dataframe

In [8]:
list_url = []

In [9]:
#Create a Dataframe for isolate the row of the links
df_link = pd.read_excel('Data-sheet-STUDENTS.xlsx',header=1,nrows=1,usecols="D:AR")
df_link = df_link.apply(str).str.replace('NaN', 'No link') 

In [10]:
#Create a loop to find inside the row the links that contain "data.worldbank"
for index, urlName in df_link.items():
    if 'data.worldbank' in urlName:
        #Extract the tax code (last part of the URL)
        extracted_url = urlName.split('/')[4].split(',')[0]
        list_url.append(extracted_url)
        
#Insert all the tax code inside a dataframe
df_link['DATA Worldbank'] = list_url

In [11]:
#Initialize the table that will contain the tax values
tab_values=[]

In [12]:
#Get the values for all the countries and for all the tax codes (src)
tab_values.append(['Country','Link','Value'])
for src in df_link['DATA Worldbank']:
    for index, iso in df['isocode'].items():
        url_src = 'http://api.worldbank.org/v2/country/'+iso+'/indicator/'+src+'?format=json&MRV=1' 
        r = rq.get(url_src)
        #GB.XPD.RSDV.GD.ZS?locations=MY-KH is not working, the link leads to nothing
        if 'GB.XPD.RSDV.GD.ZS?locations=MY-KH' in url_src:
            continue
        if r.status_code == 200:
            data = r.json() 
            if data[1] is None:    
                tax_value = 0
            else:
                tax_value = data[1][0]['value']
            #Insert the values inside the table that will show the isocode,the tax code and the value
            tab_values.append([iso,src,tax_value])
        else:
            print('no')

In [14]:
#Create a dataframe with all the values, tax codes and isocodes
df_values = pd.DataFrame(tab_values)

In [22]:
#For each line of the dataframe that contains the tax values, we'll first check if the rows of each column are empty
#Then, if it is empty it'll take the correspond value from the df_values to the df
for index, line in df_values[1].items():
    if "GC.TAX.IMPT.ZS" in df_values[1][index]:
        if pd.isnull(df.loc[index,'IMF WB\nCustoms and other import duties (% of tax revenue) 2016']):
            df.loc[index%68,'IMF WB\nCustoms and other import duties (% of tax revenue) 2016'] = df_values.at[index,2]
    elif "FB.AST.NPER.ZS" in df_values[1][index]:
        #if pd.isnull(df.loc[index,'WB IMF Non-performing loans to total gross loans ratio (%)']):
             df.loc[index%68,'WB IMF Non-performing loans to total gross loans ratio (%)'] = df_values.at[index,2]
    elif "IS.RRS.GOOD.MT.K6" in df_values[1][index]:
        #if pd.isnull(df.loc[index,'UIC Rail freight (mn ton x km travelled)']):
            df.loc[index%68,'UIC Rail freight (mn ton x km travelled)'] = df_values.at[index,2]
    elif "IS.AIR.GOOD.MT.K1" in df_values[1][index]:
        #if pd.isnull(df.loc[index,'ICAO air freight (mn ton x km travelled)']):
            df.loc[index%68,'ICAO air freight (mn ton x km travelled)'] = df_values.at[index,2]
    elif "NE.CON.PRVT.PP.CD" in df_values[1][index]:
       # if pd.isnull(df.loc[index,'WB Household final consumption expenditure, PPP bn USD']):
            df.loc[index%68,'WB Household final consumption expenditure, PPP bn USD'] = df_values.at[index,2]
    elif "GB.XPD.RSDV.GD.ZS?locations=MY-KH" in df_values[1][index]:
       # if pd.isnull(df.loc[index,'WB UNESCO R&D Expenditure % of GDP']):
            df.loc[index%68,'WB UNESCO R&D Expenditure % of GDP'] = df_values.at[index,2]
    elif "SP.POP.SCIE.RD.P6" in df_values[1][index]:
        #if pd.isnull(df.loc[index,'WB UNESCO Researchers p. mn inhabitants']):
            df.loc[index%68,'WB UNESCO Researchers p. mn inhabitants'] = df_values.at[index,2]
    elif "BX.KLT.DINV.WD.GD.ZS" in df_values[1][index]:
       # if pd.isnull(df.loc[index,'IFDI Net Inflows (% of GDP)']):
            df.loc[index%68,'IFDI Net Inflows (% of GDP)'] = df_values.at[index,2]
    elif "SH.XPD.CHEX.PP.CD" in df_values[1][index]:
       # if pd.isnull(df.loc[index,'WHO Current Health Expenditure p.c., PPP USD']):
            df.loc[index%68,'WHO Current Health Expenditure p.c., PPP USD'] = df_values.at[index,2]
    elif "GB.XPD.RSDV.GD.ZS" in df_values[1][index]:
       # if pd.isnull(df.loc[index,'WB Research Expenditure % of GDP']):
            df.loc[index%68,'WB Research Expenditure % of GDP'] = df_values.at[index,2]
    elif "IP.TMK.RSCT" in df_values[1][index]:
       # if pd.isnull(df.loc[index,'WIPO Trademark applications']):
            df.loc[index%68,'WIPO Trademark applications'] = df_values.at[index,2]
    elif "IP.PAT.RESD" in df_values[1][index]:
        #if pd.isnull(df.loc[index,'WIPO Patent Applications']):
            df.loc[index%68,'WIPO Patent Applications'] = df_values.at[index,2]

In [23]:
df[['isocode','IMF WB\nCustoms and other import duties (% of tax revenue) 2016','WB IMF Non-performing loans to total gross loans ratio (%)','UIC Rail freight (mn ton x km travelled)','ICAO air freight (mn ton x km travelled)','WB Household final consumption expenditure, PPP bn USD','WB UNESCO R&D Expenditure % of GDP','WB UNESCO Researchers p. mn inhabitants','IFDI Net Inflows (% of GDP)','WHO Current Health Expenditure p.c., PPP USD','WB Research Expenditure % of GDP','WIPO Trademark applications','WIPO Patent Applications']]

,isocode,IMF WB Customs and other import duties (% of tax revenue) 2016,WB IMF Non-performing loans to total gross loans ratio (%),UIC Rail freight (mn ton x km travelled),ICAO air freight (mn ton x km travelled),"WB Household final consumption expenditure, PPP bn USD",WB UNESCO R&D Expenditure % of GDP,WB UNESCO Researchers p. mn inhabitants,IFDI Net Inflows (% of GDP),"WHO Current Health Expenditure p.c., PPP USD",WB Research Expenditure % of GDP,WIPO Trademark applications,WIPO Patent Applications
1,AFG,35.203141,12.201830,0.000000e+00,29.559349,5.214309e+10,NaN,0.00000,0.718898,162.781158,0.00000,0.0,0.0
2,ALB,2.216459,13.232533,4.600000e+01,0.000000,2.442981e+10,NaN,156.10110,8.015508,759.666984,0.15412,1232.0,14.0
3,ARM,6.275395,5.432318,6.894600e+02,0.000000,2.464601e+10,NaN,0.00000,2.044111,876.856857,0.22770,2685.0,107.0
4,AZE,7.571847,0.000000,4.633000e+03,44.088600,1.018162e+11,NaN,0.00000,2.988948,1193.058830,0.18521,2817.0,144.0
5,BHR,27.386091,0.000000,0.000000e+00,420.983750,2.589187e+10,NaN,368.90235,4.014072,1866.297320,0.10116,253.0,8.0
6,BGD,28.923008,8.397908,7.100000e+02,63.818152,4.686410e+11,NaN,0.00000,1.072976,90.598404,0.00000,9247.0,61.0
7,BLR,8.200619,12.852194,4.853800e+04,1.900400,1.270644e+11,NaN,0.00000,2.472072,1151.408849,0.58716,4453.0,434.0
8,BTN,2.840501,8.417308,0.000000e+00,0.690916,3.833999e+09,NaN,0.00000,0.104527,293.110022,0.00000,16.0,4.0
9,BIH,0.000155,10.046275,1.114327e+03,0.000087,3.148712e+10,NaN,463.89852,2.454262,1123.425425,0.19951,742.0,87.0
10,BRN,0.000000,3.538399,0.000000e+00,129.352000,4.723443e+09,NaN,283.43110,3.770129,1812.413646,0.03701,176.0,8.0
